Inspired by the FocuStat blog post: https://www.mn.uio.no/math/english/research/projects/focustat/the-focustat-blog%21/gaudisquare.html

# MCMC - Magic $4 \times 4$ Square

Is it possible that a $4 \times 4$ square have all rows that sum to 34, all columns sums to 34 and the two diagonals sums to 34, only using the numbers 1, 2, .., 15, 16? Answer yes. The answer, to the questions that was not asked, is that it's hard to find one right away since the board has 16! ($\approx 2\cdot 10^{13}$) possible combinations to place the numbers. So, how can we solve such a problem? We can use MCMC!

We can define the sum of rows and column as 

$$r_i(x) = \sum_j x[i,j] \quad \text{ and } \quad c_i(x) = \sum_j x[j,i]$$

We want these sums to be equal to 34, so we define the functions:

$$R(x) = \sum_i|r_i(x) - 34| = 0 \quad \text{ and } \quad C(x) = \sum_i |c_i(x) - 34| = 0$$

where i = 1,2,3,4. We also want to calculate the diagonals. The diagonals is calculated in a similar fashion, but now wrt the two diagonals.

In other words, we want our R(x) = 0, C(x) = 0 and D(x) = 0. We can accomplish this by using a "nice" function that Nils gave us:

$$f_{\lambda}(x) = c \cdot e^{-\lambda\{Q(x)\}}$$

where Q(x) = R(x) + C(x) + D(x) and $\lambda$ is a postive fine-tuning parameter. _c_ is a constant that is almost impossible to solve (Nils told us aboute a norwegian nobel price winner who got his price for approximating this c).

To make it possbile to find any magic board (3x3, 5x5 or 10x10) I contructed the following class:

In [4]:
import numpy as np
import scipy.stats as stats
import time

class MagicBoard:
    
    '''
    input:
        n: 
            dimension of our n times n matrix (with the unique values 1, 2, ..., n*n)
        sim: 
            number of itterations we want our MCMC to maximumly run for 
        l: 
            our lambda, default=1.234
        shuffle_basic: 
            if we want a basic board of values 1, 2, 3, ..., n*n  or 
            if we want our values to be in random order
    
    We get a nice, magic sqaure with dimensions n times n by running the "find_magic" function.
    '''
    
    def __init__(self, n=4, sim=10**5, l=1.234, shuffle_basic=True):
        self.n = n  # number of rows/columns (n x n matrix)
        self.l = l  # lambda function
        self.sim = sim
        self.start_board = self.create_basic_board()
        self.wanted_value = np.mean(sum(self.start_board))
        if shuffle_basic == True:  # shuffle up my basic board too get random boards each time
            for i in range(10**3):
                self.start_board = self.new_board(self.start_board)
        
        self.board_row_and_column = None
        self.complete_board = None
        
    def find_magic(self):
        '''
        Input:
            row_and_col_first:
                If we want to find a board with only all rows 
                and all columns sums equal to our wanted number
        return a magic square by using MCMC.
        '''
        
        start = time.time()
        self.complete_board = self.mcmc(self.start_board)
        end = time.time()
        print(f"Time taken: {round((end-start)/60, 4)}min")
        return self.complete_board
       
            
    def create_basic_board(self):
        '''
        create a board on the form (e.g)
        1 2 3
        4 5 6
        7 8 9
        so n represent number of columns and number of rows
        '''     
        return np.arange(self.n*self.n).reshape(self.n, -1)+1
    
    def new_board(self, board):
        '''
        Takes in a board and propose a change
        by changing place of two random positions
        '''
        new_board = np.copy(board)

        row1, column1, row2, column2 = self.random_pos(board.shape)

        pos1 = board[row1, column1]
        pos2 = board[row2, column2]

        new_board[row1, column1], new_board[row2, column2] = pos2, pos1
        return new_board
    
    def random_pos(self, shape):
        '''
        Two random positions that can swich places
        '''

        row1 = round(stats.uniform.rvs(0,shape[0]-1))
        column1 = round(stats.uniform.rvs(0,shape[1]-1))

        row2 = round(stats.uniform.rvs(0,shape[0]-1))
        column2 = round(stats.uniform.rvs(0,shape[1]-1))

        if row1 == row2 and column1 == column2:
            row1, column1, row2, column2 = self.random_pos(shape)

        return row1, column1, row2, column2
    
    def Q(self, x):
        '''
        feed in a board and get the sum of difference (MAE) for
        rows + columns + diagonal.
        '''
        rows = np.sum([np.abs(np.sum(ri) - self.wanted_value) for ri in x[:,]])
        columns = np.sum([np.abs(np.sum(ci) - self.wanted_value) for ci in np.transpose(x[:,])])
        diag = np.abs(np.sum(x.diagonal()) - self.wanted_value) + np.abs(np.sum(np.fliplr(x).diagonal()) - self.wanted_value)
        return int(rows+columns+diag)
    
    def mcmc(self, start_board):
        '''
        Find a magic board using MCMC
        '''
        mcmc = [0]*self.sim
        mcmc[0] = start_board
        u1 = stats.uniform.rvs(size=self.sim)

        for i in range(1, self.sim, 1):
            old_board = mcmc[i-1]
            prop_board = self.new_board(old_board)
            prop_func = self.Q(prop_board)
            old_func = self.Q(old_board)
            pracc = min([1, np.exp(-self.l*(prop_func - old_func))])

            ok = 1*(u1[i] <= pracc)
            mcmc[i] = ok*prop_board + (1-ok)*old_board

            if prop_func == 0:
                print(f"FOUND ONE!!!")
                print(f"Number of itterations: {i}")
                mcmc[-1] = prop_board
                break

        return mcmc[-1]

First a 3x3 magic board:

In [5]:
magic_3 = MagicBoard(n=3, sim=10**5)
magic_3.wanted_value

15.0

In [6]:
magic_3.find_magic()

FOUND ONE!!!
Number of itterations: 785
Time taken: 0.0056min


array([[4, 3, 8],
       [9, 5, 1],
       [2, 7, 6]])

Then a 4x4 magic board:

In [7]:
magic_4 = MagicBoard(n=4, sim=10**5)
magic_4.wanted_value

34.0

In [8]:
magic_4.find_magic()

FOUND ONE!!!
Number of itterations: 7406
Time taken: 0.0558min


array([[ 6,  9, 12,  7],
       [13,  3,  2, 16],
       [ 4, 14, 15,  1],
       [11,  8,  5, 10]])

It only took a couple of seconds to find a 4x4 magic board. We can also try to finder even larger boeards. Let's try to find a 5x5 magic board:

In [9]:
magic_5 = MagicBoard(n=5, sim=10**6)
magic_5.wanted_value

65.0

In [10]:
magic_5.find_magic()

FOUND ONE!!!
Number of itterations: 119683
Time taken: 0.9022min


array([[11, 18, 12, 15,  9],
       [ 6, 19,  5, 13, 22],
       [10,  3, 20,  7, 25],
       [17,  2, 24, 14,  8],
       [21, 23,  4, 16,  1]])

We can also try to find a 6x6:

In [11]:
magic_6 = MagicBoard(n=6, sim=10**6)
magic_6.wanted_value

111.0

In [12]:
magic_6.find_magic()

FOUND ONE!!!
Number of itterations: 52760
Time taken: 0.428min


array([[30, 28,  6, 22, 21,  4],
       [ 3, 24, 20, 18, 32, 14],
       [ 2, 27, 10, 12, 25, 35],
       [16, 19, 29, 31,  1, 15],
       [34,  8, 13, 11,  9, 36],
       [26,  5, 33, 17, 23,  7]])

A 7x7 magic board:

In [13]:
magic_7 = MagicBoard(n=7, sim=10**7)
magic_7.wanted_value

175.0

In [14]:
magic_7.find_magic()

FOUND ONE!!!
Number of itterations: 853253
Time taken: 7.2637min


array([[ 4, 27, 24,  2, 36, 39, 43],
       [19, 42, 20, 11, 21, 30, 32],
       [29, 37, 10, 28,  5, 44, 22],
       [13, 15, 47, 34,  3, 23, 40],
       [31, 38,  8, 35, 45, 12,  6],
       [33,  9, 25, 48, 16, 26, 18],
       [46,  7, 41, 17, 49,  1, 14]])

Can also try a 8x8 magic board:

In [15]:
magic_8 = MagicBoard(n=8, sim=10**7)
magic_8.wanted_value

260.0

In [16]:
magic_8.find_magic()

FOUND ONE!!!
Number of itterations: 8883726
Time taken: 83.8515min


array([[23, 48, 12, 18, 51, 36, 61, 11],
       [17, 27, 14, 37, 30, 38, 41, 56],
       [26, 32, 58, 34,  8, 24, 16, 62],
       [52,  2, 46, 47,  6, 59, 33, 15],
       [ 7, 35,  1, 64, 55, 28, 10, 60],
       [44, 57, 45, 13,  3, 19, 50, 29],
       [42, 20, 63, 43, 53, 25,  9,  5],
       [49, 39, 21,  4, 54, 31, 40, 22]])

As you can see, my solution is not very optimized (very slow), so there is probably easy to make it work better.